In [89]:
import pandas as pd

# Path to CSV file
spreadspoke_scores_file = "../CSV_Files/spreadspoke_scores.csv"
nfl_teams_file = "../CSV_Files/nfl_teams.csv"
# Read CSV file into DataFrame
spreadspoke_scores_df = pd.read_csv(spreadspoke_scores_file)
nfl_teams_df = pd.read_csv(nfl_teams_file)

# Data inspection
spreadspoke_scores_df

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,9/2/1966,1966,1,False,Miami Dolphins,14,23,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71.0,NaN
1,9/3/1966,1966,1,False,Houston Oilers,45,7,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70.0,NaN
2,9/4/1966,1966,1,False,San Diego Chargers,27,7,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82.0,NaN
3,9/9/1966,1966,2,False,Miami Dolphins,14,19,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78.0,NaN
4,9/10/1966,1966,1,False,Green Bay Packers,24,3,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13796,1/21/2024,2023,Division,True,Buffalo Bills,24,27,Kansas City Chiefs,BUF,-2.5,46,Highmark Stadium,False,25.0,11.0,67.0,NaN
13797,1/21/2024,2023,Division,True,Detroit Lions,31,23,Tampa Bay Buccaneers,DET,-6.0,49.5,Ford Field,False,72.0,0.0,NaN,indoor
13798,1/28/2024,2023,Conference,True,Baltimore Ravens,10,17,Kansas City Chiefs,BAL,-4.5,44,M&T Bank Stadium,False,47.0,7.0,83.0,NaN
13799,1/28/2024,2023,Conference,True,San Francisco 49ers,34,31,Detroit Lions,SF,-7.5,53.5,Levi's Stadium,False,69.0,5.0,55.0,NaN


In [ ]:
nfl_teams_df

In [90]:
# Data cleaning and filtering
last_5_years_df = spreadspoke_scores_df[spreadspoke_scores_df['schedule_season'] >= 2019]
clean_df = last_5_years_df.drop(columns=['stadium', 'stadium_neutral', 'weather_temperature', 'weather_wind_mph',
                              'weather_humidity', 'weather_detail', 'spread_favorite', 'over_under_line'])
clean_df


,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id
12411,9/5/2019,2019,1,False,Chicago Bears,3,10,Green Bay Packers,CHI
12412,9/8/2019,2019,1,False,Arizona Cardinals,27,27,Detroit Lions,DET
12413,9/8/2019,2019,1,False,Carolina Panthers,27,30,Los Angeles Rams,LAR
12414,9/8/2019,2019,1,False,Cleveland Browns,13,43,Tennessee Titans,CLE
12415,9/8/2019,2019,1,False,Dallas Cowboys,35,17,New York Giants,DAL
...,...,...,...,...,...,...,...,...,...
13796,1/21/2024,2023,Division,True,Buffalo Bills,24,27,Kansas City Chiefs,BUF
13797,1/21/2024,2023,Division,True,Detroit Lions,31,23,Tampa Bay Buccaneers,DET
13798,1/28/2024,2023,Conference,True,Baltimore Ravens,10,17,Kansas City Chiefs,BAL
13799,1/28/2024,2023,Conference,True,San Francisco 49ers,34,31,Detroit Lions,SF


In [98]:
# Save 'team_favorite_id' and count which is in abbreviation_list
team_favorite_count_list = []
team_key_df = pd.DataFrame(nfl_teams_df)
team_key_df['Abbreviation'] = team_key_df['Abbreviation'].replace('LV', 'LVR')

for team_abbreviation in nfl_teams_df['Abbreviation']:
    count = clean_df[clean_df['team_favorite_id'] == team_abbreviation].shape[0]
    team_favorite_count_list.append((team_abbreviation, count))

# Sort count from Highest 
team_favorite_count_list.sort(key=lambda x: x[1], reverse=True)

team_favorite_count_list

[('KC', 87),
 ('SF', 73),
 ('BUF', 69),
 ('BAL', 66),
 ('DAL', 62),
 ('TB', 60),
 ('GB', 59),
 ('PHI', 58),
 ('LAR', 55),
 ('NO', 54),
 ('LAC', 53),
 ('SEA', 51),
 ('NE', 50),
 ('CLE', 49),
 ('MIN', 49),
 ('IND', 43),
 ('TEN', 42),
 ('PIT', 41),
 ('CIN', 40),
 ('MIA', 35),
 ('DEN', 33),
 ('ATL', 31),
 ('DET', 29),
 ('ARI', 26),
 ('CHI', 24),
 ('JAX', 24),
 ('HOU', 23),
 ('CAR', 21),
 ('WAS', 20),
 ('NYJ', 15),
 ('NYG', 14),
 ('LV', 1)]

In [92]:
# Calculate win percentage for each team
team_win_percentage_list = []

for team_abbreviation in nfl_teams_df['Name']:
    total_games = 0
    wins = 0

    for index, row in clean_df.iterrows():
        if row['team_home'] == team_abbreviation:
            total_games += 1
            if row['score_home'] > row['score_away']:
                wins += 1
        elif row['team_away'] == team_abbreviation:
            total_games += 1
            if row['score_away'] > row['score_home']:
                wins += 1
    
    # Calculate win percentage
    if total_games != 0:
        win_percentage = round((wins / total_games) * 100, 2)
    else:
        win_percentage = 0
    
    team_win_percentage_list.append((team_abbreviation, win_percentage))

# Sort teams by win percentage from highest to lowest
team_win_percentage_list.sort(key=lambda x: x[1], reverse=True)

team_win_percentage_list

[('Kansas City Chiefs', 77.78),
 ('Buffalo Bills', 68.48),
 ('Green Bay Packers', 65.56),
 ('San Francisco 49ers', 65.26),
 ('Baltimore Ravens', 65.17),
 ('New Orleans Saints', 59.3),
 ('Tampa Bay Buccaneers', 58.7),
 ('Dallas Cowboys', 58.62),
 ('Los Angeles Rams', 56.67),
 ('Seattle Seahawks', 56.32),
 ('Pittsburgh Steelers', 55.81),
 ('Philadelphia Eagles', 55.06),
 ('Minnesota Vikings', 53.49),
 ('Tennessee Titans', 53.41),
 ('Miami Dolphins', 51.76),
 ('Cleveland Browns', 51.16),
 ('New England Patriots', 48.24),
 ('Indianapolis Colts', 47.62),
 ('Cincinnati Bengals', 47.19),
 ('Las Vegas Raiders', 47.06),
 ('Los Angeles Chargers', 42.86),
 ('Detroit Lions', 39.53),
 ('Atlanta Falcons', 38.55),
 ('Denver Broncos', 38.55),
 ('Arizona Cardinals', 38.1),
 ('Chicago Bears', 38.1),
 ('Houston Texans', 37.93),
 ('New York Giants', 35.29),
 ('Washington Commanders', 35.29),
 ('Jacksonville Jaguars', 34.12),
 ('New York Jets', 32.53),
 ('Carolina Panthers', 28.92)]

In [93]:
# team_favorite_count_list를 DataFrame으로 변환
favorite_df = pd.DataFrame(team_favorite_count_list, columns=['Abbreviation', 'Count'])

# team_win_percentage_list를 DataFrame으로 변환
win_percentage_df = pd.DataFrame(team_win_percentage_list, columns=['Name', 'Win Rate'])

# 'Name'을 기준으로 'Abbreviation' 값을 가져오기 위한 딕셔너리 생성
abbreviation_dict = {'Kansas City Chiefs': 'KC', 'Buffalo Bills': 'BUF', 'Green Bay Packers': 'GB',
                     'San Francisco 49ers': 'SF', 'Baltimore Ravens': 'BAL', 'New Orleans Saints': 'NO',
                     'Tampa Bay Buccaneers': 'TB', 'Dallas Cowboys': 'DAL', 'Los Angeles Rams': 'LAR',
                     'Seattle Seahawks': 'SEA', 'Pittsburgh Steelers': 'PIT', 'Philadelphia Eagles': 'PHI',
                     'Minnesota Vikings': 'MIN', 'Tennessee Titans': 'TEN', 'Miami Dolphins': 'MIA',
                     'Cleveland Browns': 'CLE', 'New England Patriots': 'NE', 'Indianapolis Colts': 'IND',
                     'Cincinnati Bengals': 'CIN', 'Las Vegas Raiders': 'LV', 'Los Angeles Chargers': 'LAC',
                     'Detroit Lions': 'DET', 'Atlanta Falcons': 'ATL', 'Denver Broncos': 'DEN',
                     'Arizona Cardinals': 'ARI', 'Chicago Bears': 'CHI', 'Houston Texans': 'HOU',
                     'New York Giants': 'NYG', 'Washington Commanders': 'WAS', 'Jacksonville Jaguars': 'JAX',
                     'New York Jets': 'NYJ', 'Carolina Panthers': 'CAR'}

# 'Name'을 기준으로 'Abbreviation' 값을 가져옴
win_percentage_df['Abbreviation'] = win_percentage_df['Name'].map(abbreviation_dict)

# Merge Dataframes
combined_df = favorite_df.merge(win_percentage_df, on='Abbreviation')

# Rearrange the DataFrame
combined_df = pd.DataFrame(combined_df, columns=['Name', 'Abbreviation', 'Count', 'Win Rate'])

combined_df

,Name,Abbreviation,Count,Win Rate
0,Kansas City Chiefs,KC,87,77.78
1,San Francisco 49ers,SF,73,65.26
2,Buffalo Bills,BUF,69,68.48
3,Baltimore Ravens,BAL,66,65.17
4,Dallas Cowboys,DAL,62,58.62
5,Tampa Bay Buccaneers,TB,60,58.70
6,Green Bay Packers,GB,59,65.56
7,Philadelphia Eagles,PHI,58,55.06
8,Los Angeles Rams,LAR,55,56.67
9,New Orleans Saints,NO,54,59.30
